---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [975]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(50)

0          03/25/93 Total time of visit (in minutes):\n
1                        6/18/85 Primary Care Doctor:\n
2     sshe plans to move as of 7/8/71 In-Home Servic...
3                 7 on 9/27/75 Audit C Score Current:\n
4     2/6/96 sleep studyPain Treatment Pain Level (N...
5                     .Per 7/06/79 Movement D/O note:\n
6     4, 5/18/78 Patient's thoughts about current su...
7     10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                          3/7/86 SOS-10 Total Score:\n
9              (4/10/71)Score-1Audit C Score Current:\n
10    (5/11/85) Crt-1.96, BUN-26; AST/ALT-16/22; WBC...
11                        4/09/75 SOS-10 Total Score:\n
12    8/01/98 Communication with referring physician...
13    1/26/72 Communication with referring physician...
14    5/24/1990 CPT Code: 90792: With medical servic...
15    1/25/2011 CPT Code: 90792: With medical servic...
16          4/12/82 Total time of visit (in minutes):\n
17         1; 10/13/1976 Audit C Score, Highest/

In [1018]:
grupo1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
grupo2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
grupo3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
grupo4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
grupo5 = '(\d{1,2}[/-][1|2]\d{3})'
grupo6 = '([1|2]\d{3})'


In [1019]:
full_grupo = '(%s|%s|%s|%s|%s|%s)' %(grupo1,grupo2,grupo3, grupo4, grupo5, grupo6)

In [1020]:
full_grupo

'((\\d{1,2}[/-]\\d{1,2}[/-]\\d{2,4})|((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\\S]*[+\\s]\\d{1,2}[,]{0,1}[+\\s]\\d{4})|(\\d{1,2}[+\\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\\S]*[+\\s]\\d{4})|((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\\S]*[+\\s]\\d{4})|(\\d{1,2}[/-][1|2]\\d{3})|([1|2]\\d{3}))'

In [1021]:
parsed_date = df.str.extractall(full_grupo)

In [1022]:
parsed_date

,,0,1,2,3,4,5,6
,match,,,,,,,
0,0,03/25/93,03/25/93,NaN,NaN,NaN,NaN,NaN
1,0,6/18/85,6/18/85,NaN,NaN,NaN,NaN,NaN
2,0,7/8/71,7/8/71,NaN,NaN,NaN,NaN,NaN
3,0,9/27/75,9/27/75,NaN,NaN,NaN,NaN,NaN
4,0,2/6/96,2/6/96,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
495,0,1979,NaN,NaN,NaN,NaN,NaN,1979
496,0,2006,NaN,NaN,NaN,NaN,NaN,2006
497,0,2008,NaN,NaN,NaN,NaN,NaN,2008


In [1023]:
#parsed_date=parsed_date.drop(parsed_date.index[[501]])

In [1024]:
parsed_date[0].unique()

array(['03/25/93', '6/18/85', '7/8/71', '9/27/75', '2/6/96', '7/06/79',
       '5/18/78', '10/24/89', '3/7/86', '4/10/71', '5/11/85', '4/09/75',
       '8/01/98', '1/26/72', '5/24/1990', '1/25/2011', '4/12/82',
       '10/13/1976', '4/24/98', '5/21/77', '7/21/98', '10/21/79',
       '3/03/90', '2/11/76', '07/25/1984', '4-13-82', '9/22/89',
       '9/02/76', '9/12/71', '10/24/86', '03/31/1985', '7/20/72',
       '4/12/87', '06/20/91', '5/12/2012', '3/15/83', '2/14/73',
       '5/24/88', '7/27/1986', '1-14-81', '7-29-75', '6/24/87', '8/14/94',
       '4/13/2002', '8/16/82', '2/15/1998', '7/15/91', '06/12/94',
       '9/17/84', '2/28/75', '11/22/75', '5/24/91', '6/13/92', '7/11/71',
       '12/26/86', '10/11/1987', '3/14/95', '12/01/73', '12/5/2010',
       '08/20/1982', '7/24/95', '8/06/83', '02/22/92', '6/28/87',
       '07/29/1994', '08/11/78', '10/29/91', '7/6/91', '1/21/87',
       '11/3/1985', '7/04/82', '4-13-89', '7/11/77', '9/36/308',
       '4/12/74', '09/19/81', '9/6/79', '12/5

In [1025]:
parsed_date

,,0,1,2,3,4,5,6
,match,,,,,,,
0,0,03/25/93,03/25/93,NaN,NaN,NaN,NaN,NaN
1,0,6/18/85,6/18/85,NaN,NaN,NaN,NaN,NaN
2,0,7/8/71,7/8/71,NaN,NaN,NaN,NaN,NaN
3,0,9/27/75,9/27/75,NaN,NaN,NaN,NaN,NaN
4,0,2/6/96,2/6/96,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
495,0,1979,NaN,NaN,NaN,NaN,NaN,1979
496,0,2006,NaN,NaN,NaN,NaN,NaN,2006
497,0,2008,NaN,NaN,NaN,NaN,NaN,2008


In [1026]:
parsed_date = parsed_date.iloc[:,0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')

In [1027]:
parsed_date

     match
0    0        03/25/93
1    0         6/18/85
2    0          7/8/71
3    0         9/27/75
4    0          2/6/96
                ...   
495  0            1979
496  0            2006
497  0            2008
498  0            2005
499  0            1980
Name: 0, Length: 501, dtype: object

In [1028]:
parsed_date = pd.Series(pd.to_datetime(parsed_date,errors='coerce'))


In [1029]:
parsed_date

     match
0    0       1993-03-25
1    0       1985-06-18
2    0       1971-07-08
3    0       1975-09-27
4    0       1996-02-06
                ...    
495  0       1979-01-01
496  0       2006-01-01
497  0       2008-01-01
498  0       2005-01-01
499  0       1980-01-01
Name: 0, Length: 501, dtype: datetime64[ns]

In [1030]:
parsed_date = parsed_date.sort_values(ascending=True).index

In [1033]:
parsed_date

MultiIndex([(  9, 0),
            ( 84, 0),
            (  2, 0),
            ( 53, 0),
            ( 28, 0),
            (474, 0),
            (153, 0),
            ( 13, 0),
            (129, 0),
            ( 98, 0),
            ...
            (235, 0),
            (464, 0),
            (253, 0),
            (427, 0),
            (231, 0),
            (141, 0),
            (186, 0),
            (161, 0),
            (413, 0),
            ( 72, 1)],
           names=[None, 'match'], length=501)

In [1034]:
#cquitar MultiIndex en pandas
rest = parsed_date.droplevel(1)

In [1036]:
rest

Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            235, 464, 253, 427, 231, 141, 186, 161, 413,  72],
           dtype='int64', length=501)

In [1042]:
rest= pd.Series(rest, index=range(0,501))


In [1043]:
rest

0        9
1       84
2        2
3       53
4       28
      ... 
496    141
497    186
498    161
499    413
500     72
Length: 501, dtype: int64

In [1003]:
parsed_date

MultiIndex([(  9, 0),
            ( 84, 0),
            (  2, 0),
            ( 53, 0),
            ( 28, 0),
            (474, 0),
            (153, 0),
            ( 13, 0),
            (129, 0),
            ( 98, 0),
            ...
            (235, 0),
            (464, 0),
            (253, 0),
            (427, 0),
            (231, 0),
            (141, 0),
            (186, 0),
            (161, 0),
            (413, 0),
            ( 72, 1)],
           names=[None, 'match'], length=501)

In [1051]:
rest = pd.Series(rest)

In [1052]:
rest

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [1053]:
rest.pop(500) 

KeyError: 500

In [1054]:
rest

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64

In [1055]:
parsed_date.values

array([(9, 0), (84, 0), (2, 0), (53, 0), (28, 0), (474, 0), (153, 0),
       (13, 0), (129, 0), (98, 0), (111, 0), (225, 0), (31, 0), (171, 0),
       (191, 0), (486, 0), (335, 0), (415, 0), (36, 0), (405, 0),
       (323, 0), (422, 0), (375, 0), (380, 0), (345, 0), (57, 0),
       (481, 0), (436, 0), (104, 0), (299, 0), (162, 0), (154, 0),
       (402, 0), (95, 0), (73, 0), (108, 0), (156, 0), (332, 0), (182, 0),
       (82, 0), (351, 0), (278, 0), (214, 0), (155, 0), (223, 0),
       (473, 0), (49, 0), (317, 0), (11, 0), (319, 0), (40, 0), (418, 0),
       (165, 0), (370, 0), (382, 0), (3, 0), (50, 0), (363, 0), (219, 0),
       (465, 0), (237, 0), (23, 0), (342, 0), (204, 0), (258, 0),
       (315, 0), (27, 0), (93, 0), (17, 0), (488, 0), (303, 0), (283, 0),
       (395, 0), (309, 0), (419, 0), (123, 0), (19, 0), (117, 0),
       (232, 0), (72, 0), (189, 0), (369, 0), (493, 0), (318, 0),
       (239, 0), (148, 0), (105, 0), (336, 0), (6, 0), (200, 0), (81, 0),
       (65, 0), (434, 

In [1056]:
def date_sorter():
    
    # Your code here
    grupo1 = '(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})'
    grupo2 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{1,2}[,]{0,1}[+\s]\d{4})'
    grupo3 = '(\d{1,2}[+\s](?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    grupo4 = '((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[\S]*[+\s]\d{4})'
    grupo5 = '(\d{1,2}[/-][1|2]\d{3})'
    grupo6 = '([1|2]\d{3})'
    
    full_grupo = '(%s|%s|%s|%s|%s|%s)' %(grupo1,grupo2,grupo3, grupo4, grupo5, grupo6)
    parsed_date = df.str.extractall(full_grupo)
    parsed_date = parsed_date.iloc[:,0].str.replace('Janaury', 'January').str.replace('Decemeber', 'December')
    parsed_date = pd.Series(pd.to_datetime(parsed_date,errors='coerce'))
    parsed_date = parsed_date.sort_values(ascending=True).index
    
    #cquitar MultiIndex en pandas
    rest = parsed_date.droplevel(1)
    rest= pd.Series(rest, index=range(0,501))
    rest = pd.Series(rest)
    rest.pop(500) 
    
    return rest # Your answer here
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Length: 500, dtype: int64